# 词向量相似度查询

首先我们下载训练好的词向量，下载地址是

* [词向量文件](https://github.com/reyoung/chinese_demo_emb/raw/master/emb.npy), 该文件是一个标准的numpy矩阵，高为词表大小，宽为词向量宽度
* [词表文件](https://github.com/reyoung/chinese_demo_emb/raw/master/word_dict.pkl),该文件是pickle文件，的key是词汇，value是词ID

这些文件的md5值为:

In [1]:
!cat download.md5

40922abc470cc54948d32c63c12e5a04  emb.npy
b84c86fcad7f44bc7bb136fea497c259  word_dict.pkl


首先，下载这些数据到本地，执行命令

In [2]:
!md5sum -s -c download.md5 || ( curl 'https://raw.githubusercontent.com/reyoung/chinese_demo_emb/master/word_dict.pkl' > word_dict.pkl; curl 'https://raw.githubusercontent.com/reyoung/chinese_demo_emb/master/emb.npy' > emb.npy )

In [3]:
! ls -lh

total 78088
-rw-r--r--  1 baidu  wheel    90B  4 17 11:16 download.md5
-rw-r--r--  1 baidu  wheel   4.9M  4 17 11:22 emb.npy
-rw-r--r--  1 baidu  wheel    33M  4 17 11:22 word_dict.pkl
-rw-r--r--  1 baidu  wheel   6.5K  4 17 12:41 词向量相似度查询.ipynb


下一步，我们载入词表文件，将词表文件转换成词ID => 词汇的映射。

In [4]:
import cPickle

with open("word_dict.pkl") as f:
    word_dict = cPickle.load(f)

word_dict_reversed = { v:k for k,v in word_dict.items() }

显示一个文本框，接受用户的输入

In [5]:
from ipywidgets import widgets
from IPython.display import display
text = widgets.Text(description=u"查找词汇:")
display(text)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe7 in position 3: ordinal not in range(128)

进而，使用`on_text_submitted`接受用户的输入。

接受用户输入后，先去查找词表和embedding中有没有用户的输入词汇，如果没有就立即报错。

如果有用户输入的词汇，那么就查询这个词汇的embedding值，并和词表中的每一个词汇计算余弦相似度。进而排序选出最相似的前十个词汇，输出。

In [6]:
import numpy
from IPython.display import clear_output

embs = numpy.load("emb.npy")

def cosine_similarity(v1, v2):
    """compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"""
    upper = numpy.dot(v1, v2)
    return upper / (numpy.linalg.norm(v1) * numpy.linalg.norm(v2))


def similiar_words(word):
    target_word_id = word_dict[word]
    target_word_emb = embs[target_word_id]
    
    all_words = []
    
    for word_id in xrange(len(embs) - 2):
        if word_id == target_word_id: continue
        word_emb = embs[word_id]
        all_words.append((cosine_similarity(word_emb, target_word_emb), word_id))
    
    all_words.sort(key=lambda x: x[0], reverse=True)
    
    print u'与"%s"相似的词汇是：' % word
    for i in xrange(10):
        sim, wid = all_words[i]
        print '\t%s\t%.4f'%(word_dict_reversed[wid], sim)

def on_text_submitted(sender):
    txt = text.value
    clear_output()
    if txt not in word_dict or word_dict[txt] >= len(embs) - 2 :
        print u"%s 不在词表或词向量中" % txt
        return
    similiar_words(txt)
    
text.on_submit(on_text_submitted)